In [1]:
import pandas as pd
import spacy 
nlp = spacy.load("de_core_news_sm")
import matplotlib.pyplot as plt
import gzip,os,glob
import numpy as np
import pickle

In [2]:
def generate_random_vector(dimensions,seed):
    arr = np.array([0] * (dimensions-seed) + [1] * int(seed/2) + [-1] * int(seed/2) )
    np.random.shuffle(arr)
    return(arr)

In [27]:
import csv
import re
textz=[]
word_vec=dict()
word_docz=dict()
dimensions=500
seed=8

with open('../data/after_nltk/df_content_after_nltk+titles.csv', newline='', encoding="utf8") as csvfile:
    reader = csv.DictReader(csvfile) 
    for row in reader:
        textz.append(row['content'].lower())
        #if int(row['id'])>5000:
        #    break
        if int(row['id'])%1000==0:
            print(row['id'])
        current_doc_wordz=dict()
        content=row['content'].lower()
        for word in re.split('[\W]', content):
            if word not in current_doc_wordz:
                current_doc_wordz[word]=1
                try:
                    word_docz[word]=word_docz[word]+1
                except:
                    word_docz[word]=1
                    word_vec[word]=generate_random_vector(dimensions,seed)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000


In [28]:
nonsingular_wordz = dict((k, v) for k, v in word_docz.items() if v > 1)

In [29]:
normalize_factor=1/sum(nonsingular_wordz.values())
for k in nonsingular_wordz:
    nonsingular_wordz[k]*=normalize_factor

In [30]:
sum(nonsingular_wordz.values())

1.0000000000002376

In [31]:
len(textz)

18143

In [32]:
doc_vec=np.ones((len(textz),dimensions))
line_id=0
for text in textz:
    for word in re.split('[\W]',text):
        if word in nonsingular_wordz:
            doc_vec[line_id] = ( word_vec[word] / (nonsingular_wordz[word]) ) + doc_vec[line_id]
    line_id=line_id+1

<i>Instead of doc_vec[text_id] = doc_vec[text_id] + word_vec[word] You will simply have a SECOND PASS loop in which You'll do something like word_vec[word] = word_vec[word] + doc_vec[doc_id]</i>

In [33]:
# # returning the document vectors back into wordvectors
# word_vec_loop =()
# for word in word_vec:
#     word_vec_loop[word] = word_vec[word] + doc_vec[line_id]

In [34]:
# # rerunning the loop with new word_vectors
# doc_vec=np.ones((len(textz),dimensions))
# line_id=0
# for text in textz:
#     for word in re.split('[\W]',text):
#         if word in nonsingular_wordz:
#             doc_vec_loop[line_id] = ( word_vec_loop[word] / (nonsingular_wordz[word]) ) + doc_vec[line_id]
#     line_id=line_id+1

In [35]:
from scipy.spatial import distance

def nearest_neighbor(query_id,database):
    query=database[query_id]
    closest = distance.cdist([query], database, 'cosine')
    return closest

In [48]:
import numpy.ma as ma

query_id=418
nns=nearest_neighbor(query_id,doc_vec)
nn_id=np.argmin(ma.masked_where(nns<0.0000000001, nns))
print(query_id,textz[query_id])
print(nn_id,nns[0][nn_id],textz[nn_id])

418 schulbucharbeit geschichtslehrbuch unterrichtspraxis trotzen zunehmend konkurrenz gelten schulbuch leitmedium geschichtsunterrichts manchmal behaupten modern schulbuecher selbstlaeufer verfuegen ueber zahlreiche methodenseiten bieten tipps unterrichtsgestaltung gut geschichtsunterricht moeglich lehrerinnen lehrer intensiv medial eigenlogik schulbuchs auseinandersetzen entstehen schulbuecher aufbauen zeichnen gutes schulbuch methoden schulbucharbeit eignen fuer reflektieren lehrlernprozesse vorliegend band geben zentrale fragen antwort erste umfassend monografie thema
6044 0.7254626125352617 schulbuch internet bildungspolitik konsequenzen schulbuch leitmedium rolle staates schulbildung pisatests setzen schulen druck internet verbunden medialkulturelle wandel zwingen reaktion bislang bestimmen klassische schulbuch leitmedium unterricht inhalte medium staat bildungskanon unterrichtsdiskurs zentral steuern einfluss schwinden durchsetzung internets massenmedium zusehends nie können schu

<h1>Compute Cosine Similarity through dot product</h1>

In [37]:
from sklearn.preprocessing import Normalizer
nrm = Normalizer('l2')

doc_vec_normed = nrm.fit_transform(doc_vec)

In [38]:
df = pd.read_pickle('../data/after_nltk/data_after_nltk.pkl')

In [39]:
# choose example 
example = query_id
title_1 = df['title'].iloc[example]
abstract_1 = textz[example]
# get example vector
vec_1 = doc_vec_normed[example,:]

# compute similarity
sim = np.dot(doc_vec_normed[example+1:,:],vec_1)

# find most similar document, see paper
title_2 = df['title'].iloc[np.argmax(sim)+example+1]
abstract_2 = textz[np.argmax(sim)+example+1]

print('\nSimilarity:')
print(np.max(sim))
print('\nTitle 1:')
print(title_1)
print('\nAbstract 1:')
print(abstract_1)
print('\nTitle 2:')
print(title_2)
print('\nAbstract 2:')
print(abstract_2)


Similarity:
0.4470446064301517

Title 1:
 grundgebiete der elektrotechnik band 2 zeitabhängige vorgänge

Abstract 1:
grundgebiete elektrotechnik band 2 zeitabhängige vorgänge band erfolgreich lehrbuches elektrotechnik behandeln zeitabhaengigen vorgaenge elektrisch magnetischen feldern sowie netzwerken stationaeren vorgaenge band beschreiben band enthaelt ergaenzende vertiefend aufgabensammlung modern lehrbuch bieten didaktisch ausgefeilt darstellung umfangreich stoffes geben leser optimale lernhilfen angabe lernziele durchgerechnet beispiele bungsaufgaben loesungen viel kontrollfragen wichtig fachausdruecke englisch version angeben bezug praxis beschreibung unterschiedlich anwendungen zahlreich fachgebieten elektrotechnik staendig herstellen dadurch motivation lernend verstaerkt buch wenden studierend elektrotechnik saemtlicher fachrichtungen technisch hochschulen universitaeten sowohl begleittext vorlesungen selbststudium eignen

Title 2:
 grundgebiete der elektrotechnik band 1 stati

<h1>K nearest Neighbors</h1>

In [49]:
# compares similarity of two abstracts through dot product

# validation_vector1 
validation_vector1 = query_id
val_abstract_1 = textz[validation_vector1]
# get validation_vector1
val_vec_1 = doc_vec_normed[validation_vector1,:]

# compute similarity of all entities
vec_temp = []
for vec in doc_vec_normed:
    i = np.dot(val_vec_1,vec)
    vec_temp.append(i)

# turn list into DataFrame
df_vec_temp = pd.DataFrame(vec_temp)

# Create new DataFrame for sorted sim-values
df_vec_temp_sorted = df_vec_temp.copy()
df_vec_temp_sorted.sort_values(by=[0],inplace = True, ascending=False)

# Get list of indices
lst_vec_temp_sorted = df_vec_temp_sorted.iloc[1:11,:].index.values.tolist()
lst_vec_temp_sorted

print(df['title'].iloc[validation_vector1])
print('\n')

for i in lst_vec_temp_sorted:
#    print('\n')
    print(df_vec_temp.iloc[i].values)
    print(df['title'].iloc[i])
#     print(textz[i])
#     print('\n')

 schulbucharbeit das geschichtslehrbuch in der unterrichtspraxis


[0.27453739]
 das schulbuch zwischen internet und bildungspolitik konsequenzen für das schulbuch als leitmedium und die rolle des staates in der schulbildung
[0.24831622]
 kursbuch geschichte neubearbeitung allgemeine ausgabe von der antike bis zur gegenwart schülerbuch
[0.19637193]
 die grenzen europas die europäische union zwischen erweiterung und überdehnung
[0.18055889]
 musikalische akustik ein handbuch
[0.16898805]
 bildungsmedien online kostenloses lehrmaterial aus dem internet marktsichtung und empirische nutzungsanalyse
[0.16492219]
 ernährungsökologie komplexen herausforderungen im bereich ernährung integrativ begegnen
[0.16461432]
 portfolio kompetenzen standards neue wege in der lehrerbildung für berufsbildende schulen 
[0.16369007]
 erfolgreich mit den großen des marketings
[0.1634436]
 die kritik stirners und die kritik an stirner deutschportugiesisches symposion im oktober 2008 an der universidade de lisb

In [47]:
np.save('../data/after_nltk/doc_vec-200.npy', doc_vec)